In [17]:
import time
import json
import csv
from selenium import webdriver
from bs4 import BeautifulSoup

# --- 1단계: 모든 햄버거 상품의 고유 ID 수집 ---
def get_burger_ids():
    """롯데리아 브랜드 페이지에 접속하여 모든 버거의 고유 ID를 수집합니다."""

    # 분기점: [크롬 브라우저 실행] -> [롯데리아 브랜드 메인 페이지 접속]
    driver = webdriver.Chrome()
    main_url = "https://www.lotteeatz.com/brand/ria"
    driver.get(main_url)

    # 분기점: [자바스크립트가 상품 목록을 로드할 때까지 대기]
    # ISMS 인증 사이트이므로, 사람처럼 행동하기 위해 넉넉히 기다립니다.
    print("페이지 로딩 및 상품 목록 확인 중...")
    time.sleep(20)

    # 분기점: [로드된 페이지 소스를 BeautifulSoup으로 분석] -> [상품 ID 추출]
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # 'prod-list' 클래스를 가진 ul 태그 안의 모든 'prod-item' li 태그를 선택
    product_list = soup.select('ul.prod-list > li.prod-item')

    burger_ids = []
    for item in product_list:
        try:
            # 각 li 태그의 onclick 속성 값(예: "goBrandDetail('REP_000741')")을 가져옴
            onclick_attr = item.get('onclick')
            if onclick_attr and 'goBrandDetail' in onclick_attr:
                # 괄호와 따옴표 안의 고유 ID만 정교하게 추출
                product_id = onclick_attr.split("'")[1]
                burger_ids.append(product_id)
        except Exception as e:
            print(f"ID 추출 중 오류 발생: {e}")

    driver.quit() # ID 수집이 끝나면 브라우저를 닫습니다.
    print(f"총 {len(burger_ids)}개의 상품 ID를 수집했습니다.")
    return burger_ids

In [18]:
# --- 2단계: 각 상품 ID별 상세 정보 수집 ---
def get_burger_details(burger_ids):
    """수집된 상품 ID 리스트를 기반으로 각 상품의 상세 정보를 크롤링합니다."""

    all_burger_data = []
    driver = webdriver.Chrome()

    # 분기점: [수집된 ID 목록을 하나씩 순회 시작]
    for i, product_id in enumerate(burger_ids):
        # 분기점: [상세 정보 페이지 URL 조합 및 접속]
        detail_url = f"https://www.lotteeatz.com/products/introductions/{product_id}"
        driver.get(detail_url)

        # '착한 크롤러'를 위한 필수 대기 시간
        time.sleep(10)

        detail_soup = BeautifulSoup(driver.page_source, 'html.parser')

        try:
            # 분기점: [상세 정보 추출 시작]
            # 1. 버거 이름
            name = detail_soup.select_one('div.prod-tit').text.strip()

            # 2. 영양소 정보 (테이블 형태)
            nutrition_info = {}
            # '영양소 정보' 제목을 찾고, 그 다음에 오는 table을 선택
            nutrition_table = detail_soup.find('div', class_='btext-tit', string='영양소 정보').find_next_sibling('div', class_='tbl-info-wrap')
            if nutrition_table:
                rows = nutrition_table.select('tbody > tr')
                for row in rows:
                    key = row.select_one('th').text.strip()
                    value = row.select_one('td').text.strip()
                    nutrition_info[key] = value

            # 3. 알러지 정보
            allergy_info = ""
            # '알러지 정보' 제목을 찾고, 그 다음에 오는 p 태그를 선택
            allergy_p = detail_soup.find('div', class_='btext-tit', string='알러지 정보').find_next_sibling('p', class_='btext')
            if allergy_p:
                allergy_info = allergy_p.text.strip()

            # 4. (추가) 원산지 정보
            origin_info = {}
            origin_div = detail_soup.find('div', class_='btext-tit', string='원산지 정보')
            if origin_div:
                origin_table = origin_div.find_next_sibling('div', class_='tbl-info-wrap')
                if origin_table:
                    rows = origin_table.select('tbody > tr')
                    for row in rows:
                        key = row.select_one('th').text.strip()
                        value = row.select_one('td').text.strip()
                        origin_info[key] = value

            # 분기점: [추출된 정보를 딕셔너리로 통합 후 최종 리스트에 추가]
            burger_data = {
                "id": product_id,
                "name": name,
                "nutrition": nutrition_info,
                "allergy": allergy_info,
                "origin": origin_info
            }
            all_burger_data.append(burger_data)

            print(f"({i+1}/{len(burger_ids)}) ✅ '{name}' 정보 수집 완료")

        except Exception as e:
            print(f"({i+1}/{len(burger_ids)}) ❌ ID '{product_id}' 정보 수집 중 오류 발생: {e}")

    driver.quit() # 모든 작업이 끝나면 브라우저를 닫습니다.
    return all_burger_data

In [19]:
# --- 3단계: 데이터 저장 ---
def save_data(data):
    """수집된 데이터를 JSON과 CSV 파일로 저장합니다."""
    # JSON 저장
    with open("lotte_eatz_burgers.json", "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료: lotte_eatz_burgers.json")

    # CSV 저장을 위해 데이터 평탄화
    csv_data = []
    for item in data:
        # 영양정보와 원산지 정보를 보기 좋게 문자열로 변환
        nutrition_str = ", ".join([f"{k}: {v}" for k, v in item['nutrition'].items()])
        origin_str = ", ".join([f"{k}: {v}" for k, v in item['origin'].items()])

        csv_data.append([
            item['id'],
            item['name'],
            nutrition_str,
            item['allergy'],
            origin_str
        ])

    # CSV 저장
    with open("lotte_eatz_burgers.csv", "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(['ID', '이름', '영양정보', '알러지정보', '원산지정보'])
        writer.writerows(csv_data)
    print("✅ CSV 파일 저장 완료: lotte_eatz_burgers.csv")

In [20]:
# --- 메인 실행 블록 ---
if __name__ == '__main__':
    # 1. 상품 ID 수집
    product_ids = get_burger_ids()

    # 2. 상세 정보 수집
    if product_ids:
        burger_details = get_burger_details(product_ids)

        # 3. 데이터 저장
        if burger_details:
            save_data(burger_details)
            print("\n🎉 모든 작업이 성공적으로 완료되었습니다!")

페이지 로딩 및 상품 목록 확인 중...
총 0개의 상품 ID를 수집했습니다.
